In [2]:
from surprise import Dataset
from surprise import Reader
from surprise import BaselineOnly, KNNBasic
from surprise import accuracy
from surprise.model_selection import KFold

In [ ]:
'''
原理说明：
选择了scikit的surprise库来进行推荐，分别使用了ALS和SGD两种优化方法。
采用的推荐模型是隐语义模型，其基本思想是通过对稀疏矩阵进行模型分解，评估出缺失项的值，以此来得到一个基本的训练模型（采用ALS和SGD对训练模型进行优化）。
然后依照此模型针对新的用户和物品数据进行评估。
'''

In [16]:
#读取文档数据
reader = Reader(line_format='user item rating timestamp', sep=',', skip_lines=1)
data = Dataset.load_from_file('./ratings.csv', reader=reader)
train_set = data.build_full_trainset()

range(0, 7120)
range(0, 14026)
['__class__', '__del__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__name__', '__ne__', '__new__', '__next__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'close', 'gi_code', 'gi_frame', 'gi_running', 'gi_yieldfrom', 'send', 'throw']


In [20]:
bsl_options_als = {'method': 'als','n_epochs': 5,'reg_u': 12,'reg_i': 5}#传递参数，采用ALS（交替最小二乘法）优化方法，迭代5次，user的正则化参数设置为12，item的正则化参数设置为5

In [21]:
bsl_options_sgd = {'method': 'sgd','n_epochs': 5}#传递参数，采用SGD(随机梯度下降)优化方法，迭代5次

In [22]:
algo_als= BaselineOnly(bsl_options=bsl_options_als)

In [23]:
algo_sgd= BaselineOnly(bsl_options=bsl_options_sgd)

In [27]:
# 采用K折交叉验证,ALS优化的预测结果
kf = KFold(n_splits=3)
for trainset, testset in kf.split(data):
    algo_als.fit(trainset)
    predictions = algo_als.test(testset)
    accuracy.rmse(predictions, verbose=True)

uid = str(505)
iid = str(11000)
# 输出uid对iid的预测结果
pred = algo_als.predict(uid, iid, r_ui=4, verbose=True)


Estimating biases using als...
RMSE: 0.8617
Estimating biases using als...
RMSE: 0.8651
Estimating biases using als...
RMSE: 0.8651
user: 505        item: 11000      r_ui = 4.00   est = 3.72   {'was_impossible': False}


In [28]:
# 采用K折交叉验证,SGD优化的预测结果
kf = KFold(n_splits=3)
for trainset, testset in kf.split(data):
    algo_sgd.fit(trainset)
    predictions = algo_sgd.test(testset)
    accuracy.rmse(predictions, verbose=True)

#假设id为505的用户对id为11000的商品的预测
uid = str(505)
iid = str(11000)
# 对结果进行预测
pred = algo_sgd.predict(uid, iid, r_ui=4, verbose=True)


Estimating biases using sgd...
RMSE: 0.8754
Estimating biases using sgd...
RMSE: 0.8745
Estimating biases using sgd...
RMSE: 0.8730
user: 505        item: 11000      r_ui = 4.00   est = 3.69   {'was_impossible': False}
